In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
sns.set(style='whitegrid')

from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant

import tensorflow as tf
import warnings
warnings.simplefilter('ignore')

In [2]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

In [4]:
# Name of the BERT model to use
model_name = 'cahya/bert-base-indonesian-522M'

# Max length of tokens
max_length = 150

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

# Load the Transformers BERT model
transformer_bert_model = TFBertModel.from_pretrained(model_name, config = config)

Downloading: 100%|████████████████████████████████████████████████████████████████████| 468/468 [00:00<00:00, 30.9kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 62.0/62.0 [00:00<00:00, 6.33kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 230k/230k [00:00<00:00, 531kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 13.9kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 545M/545M [02:28<00:00, 3.67MB/s]
Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

In [11]:
abbertmodel = tf.keras.models.load_model('model_save')
abbertmodel.summary()

Model: "AbBERT_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 150)]             0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  110617344 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             150, 768),                          
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                      

In [22]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def hapus_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def hapus_spesial_karakter(text):
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\'," ")
    text = text.encode('ascii', 'replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    return text.replace("http://", " ").replace("https://", " ")

def hapus_angka(text):
    return  re.sub(r"\d+", "", str(text))

def hapus_tandabaca(text):
    return text.translate(str.maketrans(dict.fromkeys(string.punctuation, ' ')))

def hapus_whitespace_LT(text):
    return text.strip()

def hapus_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def word_tokenize_wrapper(text):
    return word_tokenize(text)

list_stopwords = stopwords.words('indonesian')

txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

list_stopwords = set(list_stopwords)

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

original_text = input('Masukkan text di sini: ')

input_text = hapus_single_char(original_text)
input_text = hapus_spesial_karakter(input_text)
input_text = hapus_angka(input_text)
input_text = hapus_tandabaca(input_text)
input_text = hapus_whitespace_LT(input_text)
input_text = hapus_whitespace_multiple(input_text)
tokenize_input = word_tokenize_wrapper(input_text)
stopword_text = stopwords_removal(tokenize_input)
sentence = ' '.join([str(item) for item in stopword_text])
sentence = sentence.lower()

# Set ke tokenizernya
x_test = tokenizer(
          text=sentence,
          add_special_tokens=True,
          max_length=max_length,
          truncation=True,
          padding='max_length', 
          return_tensors='tf',
          return_token_type_ids = False,
          return_attention_mask = True,
          verbose = True)

label_predicted_test = abbertmodel.predict(
    x={'input_ids': x_test['input_ids']},
    verbose=1
)

label_predicted_test['Kategori']

mylist1 = label_predicted_test
todf=pd.DataFrame.from_dict(mylist1['Kategori']) 
probabilitas = todf.sort_values(by = 0, axis = 1, ascending=False)

label_pred_max=[np.argmax(i) for i in label_predicted_test['Kategori']]

print(sentence)
print('\n')
print('KETERANGAN LABEL')
print('Bisnis: 0 '+'Hukum: 1 '+'Kesehatan: 2 '+'Komputer: 3 '+'Komunikasi: 4 '+'Matematika: 5 '+'Pendidikan: 6 '+'Pertanian: 7 '+'Teknik: 8')
print('\n')
print(f"Hasil Probabilitas: ")
print(probabilitas)
print('\n')
print(f"Prediksi Abstrak Kategori: ")
if label_pred_max == [0]:
    print(str(label_pred_max) + " Bisnis")
elif label_pred_max == [1]:
    print(str(label_pred_max) + " Hukum")
elif label_pred_max == [2]:
    print(str(label_pred_max) + " Kesehatan")
elif label_pred_max == [3]:
    print(str(label_pred_max) + " Komputer")
elif label_pred_max == [4]:
    print(str(label_pred_max) + " Komunikasi")
elif label_pred_max == [5]:
    print(str(label_pred_max) + " Matematika")
elif label_pred_max == [6]:
    print(str(label_pred_max) + " Pendidikan")
elif label_pred_max == [7]:
    print(str(label_pred_max) + " Pertanian")
elif label_pred_max == [8]:
    print(str(label_pred_max) + " Teknik")
else:
    print('Not Classify')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Masukkan text di sini: abcd
1/1 [==============================] - 10s 10s/step
abcd


KETERANGAN LABEL
Bisnis: 0 Hukum: 1 Kesehatan: 2 Komputer: 3 Komunikasi: 4 Matematika: 5 Pendidikan: 6 Pertanian: 7 Teknik: 8


Hasil Probabilitas: 
          5         2        4         0         3         1         6   
0  0.988051  0.003052  0.00282  0.002196  0.001667  0.001116  0.000457  \

          8         7  
0  0.000406  0.000235  


Prediksi Abstrak Kategori: 
[5] Matematika


In [161]:
probabilitas.iloc[:, 0:2]

,5,2
0,0.988051,0.003052


In [121]:
kategorinya=probabilitas.columns.tolist()

In [122]:
probabilitasnya=probabilitas.values.tolist()[0]

In [145]:
hasil_probsnya = np.column_stack((kategorinya,probabilitasnya))
print(hasil_probsnya)
print(type(hasil_probsnya))

[[5.00000000e+00 9.88050759e-01]
 [2.00000000e+00 3.05172428e-03]
 [4.00000000e+00 2.82046781e-03]
 [0.00000000e+00 2.19618226e-03]
 [3.00000000e+00 1.66703272e-03]
 [1.00000000e+00 1.11622212e-03]
 [6.00000000e+00 4.56916634e-04]
 [8.00000000e+00 4.05893894e-04]
 [7.00000000e+00 2.34788386e-04]]
<class 'numpy.ndarray'>


In [171]:
hasil_probsnya[:,0] = hasil_probsnya[:,0].astype(int)
print(hasil_probsnya)

[[5. 0.]
 [2. 0.]
 [4. 0.]
 [0. 0.]
 [3. 0.]
 [1. 0.]
 [6. 0.]
 [8. 0.]
 [7. 0.]]


In [141]:
# create data types: two strings of length 10 and float
dt = np.dtype('int, float')

# convert dat to an array
my_arr2 = np.array(hasil_probsnya, dt)

In [142]:
print(my_arr2)

[[(5, 5.00000000e+00) (0, 9.88050759e-01)]
 [(2, 2.00000000e+00) (0, 0.00000000e+00)]
 [(4, 4.00000000e+00) (0, 2.82046781e-03)]
 [(0, 0.00000000e+00) (0, 2.19618226e-03)]
 [(3, 3.00000000e+00) (0, 1.66703272e-03)]
 [(1, 1.00000000e+00) (0, 1.11622212e-03)]
 [(6, 6.00000000e+00) (0, 4.56916634e-04)]
 [(8, 8.00000000e+00) (0, 4.05893894e-04)]
 [(7, 7.00000000e+00) (0, 2.34788386e-04)]]


In [77]:
transform = probabilitas.transpose()
transform3 = pd.DataFrame(np.vstack([probabilitas.columns, probabilitas]))
transform3 = transform3.transpose()
transform3[0] = transform3[0].astype(int)
transform3

,0,1
0,5,0.988051
1,2,0.003052
2,4,0.002820
3,0,0.002196
4,3,0.001667
5,1,0.001116
6,6,0.000457
7,8,0.000406
8,7,0.000235


In [162]:
mylist_transform = transform3.values.tolist()

In [163]:
mylist_transform

[[5.0, 0.9880507588386536],
 [2.0, 0.003051724284887314],
 [4.0, 0.002820467809215188],
 [0.0, 0.0021961822640150785],
 [3.0, 0.0016670327167958021],
 [1.0, 0.0011162221198901534],
 [6.0, 0.0004569166339933872],
 [8.0, 0.0004058938939124346],
 [7.0, 0.00023478838556911796]]

In [208]:
my_array3 = np.asarray(mylist_transform)

In [209]:
my_array3

array([[5.00000000e+00, 9.88050759e-01],
       [2.00000000e+00, 3.05172428e-03],
       [4.00000000e+00, 2.82046781e-03],
       [0.00000000e+00, 2.19618226e-03],
       [3.00000000e+00, 1.66703272e-03],
       [1.00000000e+00, 1.11622212e-03],
       [6.00000000e+00, 4.56916634e-04],
       [8.00000000e+00, 4.05893894e-04],
       [7.00000000e+00, 2.34788386e-04]])

In [210]:
np.trunc(my_array3[:,0])

array([5., 2., 4., 0., 3., 1., 6., 8., 7.])

In [211]:
my_array3

array([[5.00000000e+00, 9.88050759e-01],
       [2.00000000e+00, 3.05172428e-03],
       [4.00000000e+00, 2.82046781e-03],
       [0.00000000e+00, 2.19618226e-03],
       [3.00000000e+00, 1.66703272e-03],
       [1.00000000e+00, 1.11622212e-03],
       [6.00000000e+00, 4.56916634e-04],
       [8.00000000e+00, 4.05893894e-04],
       [7.00000000e+00, 2.34788386e-04]])

In [213]:
arrayku = np.array(mylist_transform)
arrayku

array([[5.00000000e+00, 9.88050759e-01],
       [2.00000000e+00, 3.05172428e-03],
       [4.00000000e+00, 2.82046781e-03],
       [0.00000000e+00, 2.19618226e-03],
       [3.00000000e+00, 1.66703272e-03],
       [1.00000000e+00, 1.11622212e-03],
       [6.00000000e+00, 4.56916634e-04],
       [8.00000000e+00, 4.05893894e-04],
       [7.00000000e+00, 2.34788386e-04]])

In [218]:
int(arrayku[0][0]) == '5' print matetika

5

In [215]:
np.around(arrayku[:, 0])

array([5., 2., 4., 0., 3., 1., 6., 8., 7.])

In [ ]:
my_around = np.around(arrayku[:, 0])

In [219]:
    my_transform = pd.DataFrame(np.vstack([probabilitas.columns, probabilitas]))
    my_transform = my_transform.transpose()
    my_transform[0] = my_transform[0].astype(int)
    my_transform

,0,1
0,5,0.988051
1,2,0.003052
2,4,0.002820
3,0,0.002196
4,3,0.001667
5,1,0.001116
6,6,0.000457
7,8,0.000406
8,7,0.000235


In [230]:
my_transform = my_transform.values.tolist()
my_transform

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [221]:
my_transform = np.array(my_transform)
my_transform

array([[5.00000000e+00, 9.88050759e-01],
       [2.00000000e+00, 3.05172428e-03],
       [4.00000000e+00, 2.82046781e-03],
       [0.00000000e+00, 2.19618226e-03],
       [3.00000000e+00, 1.66703272e-03],
       [1.00000000e+00, 1.11622212e-03],
       [6.00000000e+00, 4.56916634e-04],
       [8.00000000e+00, 4.05893894e-04],
       [7.00000000e+00, 2.34788386e-04]])

In [229]:
my_transform.toarray

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [236]:
    mytest = pd.DataFrame(np.vstack([probabilitas.columns, probabilitas]))
    mytest = mytest.transpose()
    mytest[0] = mytest[0].astype(int)
    mytest = mytest.values.tolist()
    mytest[0:5]

[1.0, 0.0011162221198901534]

In [238]:
mytest[0][1]

0.9880507588386536

In [242]:
mytest

[[5.0, 0.9880507588386536],
 [2.0, 0.003051724284887314],
 [4.0, 0.002820467809215188],
 [0.0, 0.0021961822640150785],
 [3.0, 0.0016670327167958021],
 [1.0, 0.0011162221198901534],
 [6.0, 0.0004569166339933872],
 [8.0, 0.0004058938939124346],
 [7.0, 0.00023478838556911796]]

In [244]:
coba = pd.DataFrame(np.vstack([probabilitas.columns, probabilitas]))
coba = coba.transpose()
coba[0] = coba[0].astype(int)
coba

,0,1
0,5,0.988051
1,2,0.003052
2,4,0.002820
3,0,0.002196
4,3,0.001667
5,1,0.001116
6,6,0.000457
7,8,0.000406
8,7,0.000235


In [256]:
category = []
for row in coba[0]:
    if row == 0 :   category.append('Business')
    elif row == 1:  category.append('Law')
    elif row == 2:  category.append('Health')
    elif row == 3:  category.append('Computer')
    elif row == 4:  category.append('Communication')
    elif row == 5:  category.append('Mathematics')
    elif row == 6:  category.append('Education')
    elif row == 7:  category.append('Agriculture')
    elif row == 8:  category.append('Engineering')
    else:          category.append('Not_Classify')

In [257]:
coba['category'] = category

In [258]:
coba

,0,1,category
0,5,0.988051,Mathematics
1,2,0.003052,Health
2,4,0.002820,Communication
3,0,0.002196,Business
4,3,0.001667,Computer
5,1,0.001116,Law
6,6,0.000457,Education
7,8,0.000406,Engineering
8,7,0.000235,Agriculture


In [252]:
coba[0]

0    5
1    2
2    4
3    0
4    3
5    1
6    6
7    8
8    7
Name: 0, dtype: int32

In [259]:
mycoba = coba.values.tolist()

In [260]:
mycoba

[[5, 0.9880507588386536, 'Mathematics'],
 [2, 0.003051724284887314, 'Health'],
 [4, 0.002820467809215188, 'Communication'],
 [0, 0.0021961822640150785, 'Business'],
 [3, 0.0016670327167958021, 'Computer'],
 [1, 0.0011162221198901534, 'Law'],
 [6, 0.0004569166339933872, 'Education'],
 [8, 0.0004058938939124346, 'Engineering'],
 [7, 0.00023478838556911796, 'Agriculture']]